In [77]:
from jwt import JWT, jwk
import hashlib
import os
import requests
import uuid
from urllib.parse import urlencode, unquote

import json

# 메서드 정의

#### URL Generator

In [133]:
from typing import Literal
import urllib.parse as url_parser
from collections import namedtuple


UrlComponents = namedtuple(typename="UrlComponents", field_names=["scheme", "netloc", "url", "params", "query", "fragment"])


def generate_url(source: Literal["upbit", "coinapi"], api_name: Literal["show_account", "order_available", "make_order", "market_code", "BTC_candle"], query: dict|None=None) -> str:
    # URL 자료 구조 정의
    URL_SOURCES = {
        "upbit": "api.upbit.com",
        "coinapi": "rest.coinapi.io",
    }

    URL_PATHS = {
        "show_account": "v1/accounts",
        "order_available": "v1/orders/chance",
        "make_order": "v1/orders",
        "market_code": "v1/orders/chance",
        "BTC_candle": "v1/ohlcv/BITSTAMP_SPOT_BTC_USD/history",

    }

    SUPPORTED_API_NAMES = URL_PATHS.keys()
    SUPPORTED_SOURCES = URL_SOURCES.keys()

    # 파라미터 검증
    if source not in SUPPORTED_SOURCES:
        raise ValueError(f'source must be "upbit" or "coinapi". input is {source}')
    
    if api_name not in SUPPORTED_API_NAMES:
        raise ValueError(f'api_name {source} not found in supported api_name.')
    
    query = "" if query is None else url_parser.urlencode(query)

    url_params = UrlComponents(
        scheme = "https",
        netloc = URL_SOURCES[source],
        url = URL_PATHS[api_name],
        params = "",
        query = query,
        fragment = "",
    )

    result_url = url_parser.urlunparse(url_params)
    return result_url


#### Query Generator

In [88]:
def generate_query(**params: str) -> dict[str: str]:
    '''
    period_id:
        Second	1SEC, 2SEC, 3SEC, 4SEC, 5SEC, 6SEC, 10SEC, 15SEC, 20SEC, 30SEC
        Minute	1MIN, 2MIN, 3MIN, 4MIN, 5MIN, 6MIN, 10MIN, 15MIN, 20MIN, 30MIN
        Hour	1HRS, 2HRS, 3HRS, 4HRS, 6HRS, 8HRS, 12HRS
        Day	1DAY, 2DAY, 3DAY, 5DAY, 7DAY, 10DAY
        Month	1MTH, 2MTH, 3MTH, 4MTH, 6MTH
        Year	1YRS, 2YRS, 3YRS, 4YRS, 5YRS
    '''

    query = {i:params[i] for i in params}    
    return query


#### API Key Generator

In [118]:
from jwt import jwk

def get_api_key(source: Literal["upbit", "coinapi"]) -> tuple[str, jwk.OctetJWK]|str:
    access_key, secret_key, jwk_secret = "", "", None

    with open("./keys.json") as f:
        keys = json.load(f)

        if source == "upbit":
            access_key = keys["upbit_access"]
            secret_key = keys["upbit_secret"].encode("utf8")

        elif source == "coinapi":
            access_key = keys["coinapi_access"]

    if source == "upbit":
        jwk_secret = jwk.OctetJWK(key=secret_key)
        return (access_key, jwk_secret)
    
    else:
        return access_key

#### header Generator

In [132]:
from jwt import JWT


def generate_header(source: Literal["upbit", "coinapi"], payload: dict|None=None):
    header = {}

    if source == "upbit":
        UPBIT_ACCESS_KEY, UPBIT_SECRET_JWK = get_api_key(source)
        payload["access_key"] = UPBIT_ACCESS_KEY

        jwt_token = JWT().encode(payload, key=UPBIT_SECRET_JWK)

        authorization = 'Bearer {}'.format(jwt_token)
        header = { 'Authorization': authorization }
    
    elif source == "coinapi":
        header = {'X-CoinAPI-Key' : get_api_key(source)}

    return header


# UPBIT

## 초기 설정

In [91]:
access_key, jwk_secret = get_api_key("upbit")

## 전체 계좌 정보 불러오기

In [122]:
payload = { 'nonce': str(uuid.uuid4()) }
res = requests.get(generate_url("upbit", "show_account", None), headers=generate_header(payload))
res.json()

{'error': {'name': 'no_authorization_ip',
  'message': 'This is not a verified IP.'}}

## 주문 가능 정보 조회

In [130]:
params = { 'market': 'KRW-BTC' }
query_string = unquote(urlencode(params, doseq=True)).encode("utf-8")

hash = hashlib.sha512()
hash.update(query_string)
query_hash = hash.hexdigest()

payload = {
    'nonce': str(uuid.uuid4()),
    'query_hash': query_hash,
    'query_hash_alg': 'SHA512',
}

res = requests.get(generate_url("upbit", "order_available", None), params=params, headers=generate_header(payload))
res.json()

{'error': {'name': 'no_authorization_ip',
  'message': 'This is not a verified IP.'}}

## 주문하기
[API docs](https://docs.upbit.com/reference/%EC%A3%BC%EB%AC%B8%ED%95%98%EA%B8%B0)

In [138]:
params = {
  'market': 'KRW-BTC', # 마켓 ID
  'side': 'bid',       # 주문 종류: ["bid": 매수, "ask": 매도]
  'ord_type': 'limit', # 주문 타입: ["limit": 지정가 주문, "price": 시장가 매수, "market": 시장가 매도]
  'price': '100.0',    # 주문 가격: KRW-BTC 마켓에서 1000 -> 1BTC당 매도 1호가가 500 KRW인 경우 2BTC가 매수됨.
  'volume': '0.01',    # 주문량
}

query_string = unquote(urlencode(params, doseq=True)).encode("utf-8")

hash = hashlib.sha512()
hash.update(query_string)
query_hash = hash.hexdigest()

payload = {
    'nonce': str(uuid.uuid4()),
    'query_hash': query_hash,
    'query_hash_alg': 'SHA512',
}

'''
200:
  uuid: 주문 고유 ID
  side: 주문 종류
  ord_type: 주문 방식
  price: 주문 당시 화폐 가격
  state: 주문 상태
  market: 마켓의 유일키
  created_at: 주문 생성 시간
  volume: 입력된 주문량
  remaining_volume: 체결 후 남은 주문 양
  reserved_fee: 수수료로 예약된 비용
  remaining_fee: 남은 수수료
  paid_fee: 지불된 수수료
  locked: 거래에 할당된 비용
  executed_volume: 체결량
  trades_count: 해당 주문에 걸린 체결 수

4XX:
  error:
    error_name: 오류 이름
    message: 오류 메시지
'''
res = requests.post(generate_url("upbit", "make_order", None), json=params, headers=generate_header(payload))
res.json()

{'error': {'name': 'no_authorization_token', 'message': '로그인이 필요합니다.'}}

# COINAPI

### 비트코인 캔들 데이터 가져오기

### 비트코인 거래량

In [136]:
import requests

query = {
    "period_id": "1HRS",
    "time_start": "2023-01-01T00:00:00",
    "time_end": "2023-01-01T23:59:59",
    "include_empty_items": "true",
}

response = requests.get(generate_url("coinapi", "BTC_candle", query=query), headers=generate_header(source="coinapi"))
response.json()

[{'time_period_start': '2023-01-01T00:00:00.0000000Z',
  'time_period_end': '2023-01-01T01:00:00.0000000Z',
  'time_open': '2023-01-01T00:00:30.1080000Z',
  'time_close': '2023-01-01T00:59:52.5600000Z',
  'price_open': 16530,
  'price_high': 16532,
  'price_low': 16507,
  'price_close': 16521,
  'volume_traded': 17.05204457,
  'trades_count': 343},
 {'time_period_start': '2023-01-01T01:00:00.0000000Z',
  'time_period_end': '2023-01-01T02:00:00.0000000Z',
  'time_open': '2023-01-01T01:02:05.8230000Z',
  'time_close': '2023-01-01T01:59:54.6410000Z',
  'price_open': 16520,
  'price_high': 16542,
  'price_low': 16519,
  'price_close': 16542,
  'volume_traded': 13.29582632,
  'trades_count': 208},
 {'time_period_start': '2023-01-01T02:00:00.0000000Z',
  'time_period_end': '2023-01-01T03:00:00.0000000Z',
  'time_open': '2023-01-01T02:00:04.6930000Z',
  'time_close': '2023-01-01T02:59:49.2110000Z',
  'price_open': 16540,
  'price_high': 16546,
  'price_low': 16532,
  'price_close': 16537,
  '

## FINANCE_DATA_READER

### 비트코인 CME 선물 가격

In [73]:
import FinanceDataReader as fdr

df = fdr.DataReader("BTC", "2023")
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-01-03,90.591003,90.618797,90.570000,90.618797,90.618797,1715
2023-01-04,90.879204,90.879204,90.879204,90.879204,90.879204,103
2023-01-05,90.751404,90.751404,90.751404,90.751404,90.751404,11
2023-01-06,90.919998,91.408302,90.919998,91.408302,91.408302,11299
2023-01-09,91.429001,91.599998,91.429001,91.599998,91.599998,3790
...,...,...,...,...,...,...
2023-06-13,91.640099,91.640099,91.375000,91.375000,91.375000,3203
2023-06-14,91.459999,91.459999,91.459999,91.459999,91.459999,179
2023-06-15,91.824997,91.824997,91.824997,91.824997,91.824997,47
